2.5 Clumpy testing errors

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [15]:
df = pd.read_csv('abalone.data', header=None, names=['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings'])
df = pd.get_dummies(df, columns=['Sex'])
N = len(df)
#df.isna().any()  # df has no NA entry

In [34]:
def p_val(t_statistic, dof):
    from scipy.stats import t 
    return 2 * t.cdf(-np.abs(t_statistic), dof)

def t_test(rho, alpha=.05):  # alpha rejection level
    x = np.random.multivariate_normal(np.zeros(2), np.array([[1, rho], [rho, 1]]), N)
    noise = pd.DataFrame(x, columns=['x1', 'x2'])
    noisy_df = pd.concat([noise, df], axis=1)
    X = noisy_df.drop('Rings', axis=1).to_numpy()
    y = noisy_df['Rings'].to_numpy()
    beta_hat = np.linalg.solve(X.T @ X, X.T @ y)
    n, d = X.shape
    S = np.sqrt(1 / (n - d) * np.linalg.norm(y - X @ beta_hat, ord=2) ** 2)
    t1 = beta_hat[0] / S / np.sqrt(np.linalg.inv(X.T @ X)[0,0])
    t2 = beta_hat[1] / S / np.sqrt(np.linalg.inv(X.T @ X)[1,1])
    r1 = p_val(t1, n-d) < alpha
    r2 = p_val(t2, n-d) < alpha
    return r1, r2, r1 & r2

In [59]:
fd_counts = np.zeros((3,7))  # false discovery counts
for i, rho in enumerate([-.9,-.8,-.4,0,.4,.8,.9]):
    for _ in range(1000):
        fd_counts[:,i] += t_test(rho)

In [60]:
fd_counts / 1000

array([[0.041, 0.053, 0.049, 0.052, 0.062, 0.043, 0.056],
       [0.049, 0.06 , 0.053, 0.056, 0.056, 0.041, 0.049],
       [0.022, 0.022, 0.007, 0.005, 0.007, 0.016, 0.027]])

Individually, both x1 and x2 are rejected $\alpha=0.05$ fraction of the time.

When x1 and x2 are more correlated, the chance that they are both rejected grows.